In [1]:
import collections
import numpy as np
import statistics
from river import base, tree, drift, utils, stats

In [2]:
class ARTE(base.Ensemble, base.Classifier):
    """Adaptive Random Tree Ensemble (ARTE) portado do MOA.
    
    Algoritmo adaptativo para fluxos de dados evolutivos de Paim e Enembreck.
    """

    def __init__(
        self,
        model: base.Classifier = None,
        n_models: int = 100,
        lambd: float = 6.0,
        drift_detector: base.DriftDetector = None,
        window_size: int = 1000,
        n_rejections: int = 5,
        seed: int = 1
    ):
        # O modelo base sugerido no original é a ARFHoeffdingTree
        # No River, usamos HoeffdingTreeClassifier como base
        self.model = model or tree.HoeffdingTreeClassifier()
        self.n_models = n_models
        self.lambd = lambd
        self.drift_detector = drift_detector or drift.ADWIN(delta=1e-3)
        self.window_size = window_size
        self.n_rejections = n_rejections
        self.seed = seed
        self._rng = np.random.RandomState(self.seed)
        
        # Inicialização dos membros conforme a estrutura AREBaseLearner do original
        self._ensemble_members = []
        for i in range(self.n_models):
            m = {
                'model': self.model.clone(),
                'detector': self.drift_detector.clone(),
                'untrained_counts': collections.defaultdict(int),
                'window_acc': utils.Rolling(stats.Mean(), window_size=self.window_size),
                'instances_trained': 0
            }
            self._ensemble_members.append(m)
            
        super().__init__(models=[m['model'] for m in self._ensemble_members])
        self._avg_window_acc = 0.0

    def learn_one(self, x, y):
        all_accs = []
        
        for m in self._ensemble_members:
            # Predição para controle de erro e lógica de rejeição
            y_pred = m['model'].predict_one(x)
            correct = (y == y_pred)
            
            # Estratégia de Regularização Adaptativa:
            # Para evitar que domínios com ruído dominem, treina no erro
            # ou após N rejeições (acertos)
            will_train = not correct
            
            if correct:
                m['untrained_counts'][y] += 1
                if self.n_rejections > 0 and m['untrained_counts'][y] >= self.n_rejections:
                    m['untrained_counts'][y] = 0
                    will_train = True
            
            if will_train:
                # Online Bagging via Poisson
                k = self._rng.poisson(self.lambd)
                if k > 0:
                    for _ in range(k):
                        m['model'].learn_one(x, y)
                        m['instances_trained'] += 1
            
            # Detecção de Drift individual
            m['detector'].update(0 if correct else 1)
            if m['detector'].drift_detected:
                self._reset_member(m)
            
            # Atualiza estatísticas da janela deslizante
            m['window_acc'].update(1 if correct else 0)
            all_accs.append(m['window_acc'].get())

        # Atualiza média global para critério de votação seletiva
        if all_accs:
            self._avg_window_acc = statistics.mean(all_accs)
            
        return self

    def predict_proba_one(self, x):
        combined_votes = collections.Counter()
        
        # O ARTE filtra votantes cuja acurácia na janela é inferior à média global
        eligible_members = [
            m for m in self._ensemble_members 
            if self.window_size == 0 or m['window_acc'].get() >= self._avg_window_acc
        ]
        
        # Fallback se ninguém estiver acima da média (ex: início do stream)
        if not eligible_members:
            eligible_members = self._ensemble_members

        for m in eligible_members:
            votes = m['model'].predict_proba_one(x)
            if votes:
                total = sum(votes.values())
                if total > 0:
                    for cls, prob in votes.items():
                        combined_votes[cls] += prob / total

        return combined_votes

    def _reset_member(self, m):
        """Reinicia o modelo e estatísticas após detecção de mudança."""
        m['model'] = self.model.clone()
        m['detector'] = self.drift_detector.clone()
        m['untrained_counts'].clear()
        m['window_acc'] = utils.Rolling(stats.Mean(), window_size=self.window_size)

In [3]:
import time
import os
import psutil
import csv
import statistics
import numpy as np
from river import metrics, tree, drift, stream

def get_memory_usage():
    """Retorna o uso de memória RAM física (RSS) em MB."""
    process = psutil.Process(os.getpid())
    return process.memory_info().rss / (1024 * 1024)

# 1. Configurações da base de teste
base_path = os.path.expanduser("~/moa/aldopaim/AdaptiveRegularizedEnsemble/datasets")
dataset_name = "ElecNorm"
filename = "elecNormNew.arff"
target_column = "class"  # Já identificado anteriormente para esta base
output_file = "resultados_arte_test.csv"

# 2. Inicialização do arquivo CSV
header = ['Dataset', 'Seed', 'Acuracia', 'Tempo_s', 'Memoria_MB', 'Instancias']
with open(output_file, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)

print(f"Iniciando Teste com ARTE no dataset: {dataset_name}")
print(f"Configuração: 100 árvores, g=100, delta=0.01, 5 Seeds")
print(f"Gravando em: {output_file}\n")

# 3. Loop de Experimentação (5 Seeds)
for s in range(1, 6):
    print(f"\n>>> Seed {s} em execução...")
    print(f"{'Instâncias':<12} | {'Acurácia':<10} | {'Tempo (s)':<10} | {'Memória (MB)':<12}")
    print("-" * 55)

    # Hiperparâmetros recomendados pelo autor
    base_tree = tree.HoeffdingTreeClassifier(grace_period=100, delta=0.01)
    adwin_detector = drift.ADWIN(delta=1e-3)
    
    # Instanciação do ARTE portado
    model = ARTE(
        model=base_tree,
        n_models=100,
        drift_detector=adwin_detector,
        seed=s
    )
    metric = metrics.Accuracy()
    
    start_time = time.perf_counter()
    count = 0
    file_path = os.path.join(base_path, filename)
    
    try:
        # Carregamento via stream preguiçosa
        dataset_stream = stream.iter_arff(file_path, target=target_column)
        
        for x, y in dataset_stream:
            # Test-then-Train
            y_pred = model.predict_one(x)
            if y_pred is not None:
                metric.update(y, y_pred)
            
            model.learn_one(x, y)
            count += 1
            
            # Print intermediário a cada 5.000 instâncias para ElecNormNew
            if count % 5000 == 0:
                elapsed = time.perf_counter() - start_time
                mem = get_memory_usage()
                print(f"{count:<12} | {metric.get():>9.2%} | {elapsed:>9.2f} | {mem:>12.2f}")

        # Finalização da Rodada
        total_elapsed = time.perf_counter() - start_time
        final_mem = get_memory_usage()
        final_acc = metric.get()
        
        print("-" * 55)
        print(f"RESULTADO SEED {s}: {final_acc:.2%}")

        # Salvamento Persistente
        with open(output_file, 'a', newline='') as f:
            writer = csv.writer(f)
            writer.writerow([dataset_name, s, f"{final_acc:.4f}", f"{total_elapsed:.2f}", f"{final_mem:.2f}", count])
            f.flush() # Força a gravação física no disco

    except Exception as e:
        print(f"Falha ao processar Seed {s}: {e}")

print("\n" + "="*55)
print(f"TESTE CONCLUÍDO. Resultados disponíveis em {output_file}")

Iniciando Teste com ARTE no dataset: ElecNorm
Configuração: 100 árvores, g=100, delta=0.01, 5 Seeds
Gravando em: resultados_arte_test.csv


>>> Seed 1 em execução...
Instâncias   | Acurácia   | Tempo (s)  | Memória (MB)
-------------------------------------------------------
5000         |    94.02% |     55.70 |       329.91
10000        |    93.41% |    115.67 |       424.91
15000        |    93.47% |    175.29 |       424.91
20000        |    93.33% |    240.85 |       433.53
25000        |    92.64% |    313.92 |       447.03
30000        |    92.40% |    385.70 |       447.03
35000        |    92.20% |    461.71 |       447.03
40000        |    92.08% |    533.70 |       447.03
45000        |    92.14% |    604.28 |       447.03
-------------------------------------------------------
RESULTADO SEED 1: 92.15%

>>> Seed 2 em execução...
Instâncias   | Acurácia   | Tempo (s)  | Memória (MB)
-------------------------------------------------------
5000         |    94.04% |     58.13 |

In [ ]:
def get_target_column_name(file_path):
    """Detecta dinamicamente o nome da última coluna (target) no ARFF."""
    last_attribute = None
    with open(file_path, 'r') as f:
        for line in f:
            line_upper = line.upper().strip()
            if line_upper.startswith('@ATTRIBUTE'):
                parts = line.split()
                if len(parts) > 1:
                    last_attribute = parts[1].replace("'", "").replace('"', '')
            if line_upper.startswith('@DATA'):
                break
    return last_attribute

def get_memory_usage():
    """Retorna o uso de memória em MB."""
    process = psutil.Process(os.getpid())
    return process.memory_info().rss / (1024 * 1024)

# 1. Configuração de Caminhos e Datasets
base_path = os.path.expanduser("~/moa/aldopaim/AdaptiveRegularizedEnsemble/datasets")
output_file = "resultados_are_river.csv"
arff_files = {
    "Airlines": "airlines.arff", "Census": "census.arff", "Connect-4": "connect-4.arff",
    "Covtype": "covtypeNorm.arff", "ElecNorm": "elecNormNew.arff", "GMSC": "GMSC.arff",
    "Keystroke": "keystroke.arff", "NOAA": "NOAA.arff", "Nomao": "nomao.arff",
    "Outdoor": "outdoor.arff", "Ozone": "ozone.arff"
}

# 2. Preparação do arquivo CSV
header = ['Dataset', 'Seed', 'Acuracia', 'Tempo_s', 'Memoria_MB', 'Instancias']
with open(output_file, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)

# 3. Loop Principal: Datasets -> Seeds (1 a 5)
print(f"{'Dataset':<12} | {'Seed':<5} | {'Acurácia':<10} | {'Tempo (s)':<10}")
print("-" * 45)

for name, filename in arff_files.items():
    file_path = os.path.join(base_path, filename)
    if not os.path.exists(file_path): 
        print(f"![AVISO] Arquivo {filename} não encontrado. Pulando base {name}.")
        continue

    target_col = get_target_column_name(file_path)
    
    # Executa 5 rodadas com seeds diferentes
    for s in range(1, 6):
        print(f"\n>>> Base: {name} | Seed: {s} | Target: {target_col}")
        print(f"{'Instâncias':<12} | {'Acurácia':<10} | {'Tempo (s)':<10} | {'Memória (MB)':<12}")
        print("-" * 55)
        
        # Reinicia Hiperparâmetros para garantir independência entre as rodadas
        base_tree = tree.HoeffdingTreeClassifier(grace_period=100, delta=0.01)
        adwin_detector = drift.ADWIN(delta=1e-3)
        
        model = ARTE(
            model=base_tree,
            n_models=100,
            drift_detector=adwin_detector,
            seed=s
        )
        metric = metrics.Accuracy()
        
        start_time = time.perf_counter()
        count = 0
        
        try:
            dataset_stream = stream.iter_arff(file_path, target=target_col)
            
            for x, y in dataset_stream:
                y_pred = model.predict_one(x)
                if y_pred is not None:
                    metric.update(y, y_pred)
                model.learn_one(x, y)
                count += 1

                # Print intermediário a cada 10.000 instâncias para acompanhar evolução
                if count % 10000 == 0:
                    elapsed = time.perf_counter() - start_time
                    mem = get_memory_usage()
                    print(f"{count:<12} | {metric.get():>9.2%} | {elapsed:>9.2f} | {mem:>12.2f}")
            
            # Fim da rodada
            total_elapsed = time.perf_counter() - start_time
            final_mem = get_memory_usage()
            final_acc = metric.get()

            print("-" * 55)
            print(f"CONCLUÍDO: {name} (Seed {s}) | Acc: {final_acc:.2%} | Tempo: {total_elapsed:.2f}s")
            
            # Salva no CSV
            with open(output_file, 'a', newline='') as f:
                writer = csv.writer(f)
                writer.writerow([name, s, f"{final_acc:.4f}", f"{total_elapsed:.2f}", f"{final_mem:.2f}", count])
                f.flush() # Força a escrita no disco agora
            
            print(f"{name:<12} | {s:<5} | {acc:>9.2%} | {elapsed:>9.2f}")
            
        except Exception as e:
            print(f"Erro ao processar {name} na Seed {s}: {e}")

print("-" * 45)
print(f"Experimento concluído! Resultados salvos em: {output_file}")